In [10]:
import tabula #Extracting tables from pdf
#from pdfminer.high_level import extract_pages, ectract_text
import os
import re
from tika import parser # pip install tika

import pandas as pd
import numpy as np

In [11]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

In [12]:
path = os.listdir("/Users/Moni/Ironhack/my_ironhack/projects/final-project/data/historico-facturas-pdf/palacet")

# Extract name of documents
files = [i for i in path if ".DS" not in i]

#for testing
#files = files[3:5]

In [25]:
df = pd.DataFrame(columns=['description', 'units', 'price', "date", "order_num"])

path = path = "../../data/historico-facturas-pdf/palacet/"

for pdf in files:
    
    order_num = pdf.replace(".pdf", "")
    
    #parser the text in the pdf (returns a dict)
    raw = parser.from_file(f"{path}{pdf}")
    
    # Extract the content (returns a str)
    content = raw['content']
    
    # He know there is only one date in the document (and the format)
    pattern = r'\b\d{2}/\d{2}/\d{2}\b'  #meets the date format
    
    # Extract the date - returns a list of one elemnt
    date = re.findall(pattern, content)[0]
    
    #Extract tables from the pdf:
    tables = tabula.read_pdf(f"{path}{pdf}", pages="all", pandas_options={'header': None})
    
    #We know last table is always resume of ticket.exclude:
    #tables = tables[:-1]
    
    #Information about products(main table) in different pages, are extracted as different tables:
    for table in tables:
        
        if table[0][0] == "IVA":
            continue
        
        table.rename(columns={0: "description", 1:"units", 2:"price"}, inplace=True)
        table["date"] = date
        table["order_num"] = order_num

        result = pd.concat([df, table])

        df = result
print("pdf tables extrated")

pdf tables extrated


In [26]:
df.sample()

,description,units,price,date,order_num
0,Bolsa de basura Bosque Verde 30L cubo mediano,1,"1,35 €",16/04/21,8135057


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 134 entries, 0 to 17
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  134 non-null    object
 1   units        129 non-null    object
 2   price        129 non-null    object
 3   date         134 non-null    object
 4   order_num    134 non-null    object
dtypes: object(5)
memory usage: 6.3+ KB


In [28]:
df[df["units"].isna()]

,description,units,price,date,order_num
0,Suavizante ropa Floral Bosque Verde concentrad...,NaN,NaN,12/12/23,16395407
0,"Toallitas limpiadoras de suelo Bosque Verde23,...",NaN,NaN,12/12/23,16395407
0,"Lejía con detergente Bosque Verde22,30 €",NaN,NaN,12/12/23,16395407
0,Bolsa de basura Bosque Verde 30L cubo mediano2...,NaN,NaN,12/12/23,16395407
0,"Friegasuelos Pino Bosque Verde líquido32,85 €",NaN,NaN,12/12/23,16395407


In [29]:
df.dropna(subset=['units'], inplace=True)

In [30]:
df.sample()

,description,units,price,date,order_num
2,Escoba fibras cruzadas Bosque Verde,1,"2,10 €",07/11/20,6947878


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129 entries, 0 to 17
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  129 non-null    object
 1   units        129 non-null    object
 2   price        129 non-null    object
 3   date         129 non-null    object
 4   order_num    129 non-null    object
dtypes: object(5)
memory usage: 6.0+ KB


In [32]:
df.sample()

,description,units,price,date,order_num
10,Papel de aluminio Bosque Verde,1,"2,90 €",16/04/21,8135057


In [33]:
df["price_eu"] = df["price"].str.replace("€","").str.replace(",",".").str.strip().astype(float)
df["unit_price"] = df["price_eu"]/df["units"]

In [34]:
df["units"] = df["units"].astype(int)

In [35]:
df = df[["description", "units", "price_eu", "unit_price", "date", "order_num"]]

In [37]:
df

,description,units,price_eu,unit_price,date,order_num
0,Friegasuelos pino Bosque Verde líquido,4,2.72,0.68,16/09/20,6598904
1,Papel multiusos resistente Bosque Verde,1,2.89,2.89,16/09/20,6598904
2,Papel higiénico doble rollo Bosque Verde,3,11.46,3.82,16/09/20,6598904
3,Lavavajillas aloe Bosque Verde concentrado,4,6.20,1.55,16/09/20,6598904
4,Suavizante ropa talco Bosque Verde concentrado,2,3.58,1.79,16/09/20,6598904
...,...,...,...,...,...,...
13,Activador del lavado ropa blanca Bosque Verde ...,1,1.60,1.6,07/11/20,6947878
14,Suavizante ropa talco Bosque Verde concentrado,2,3.58,1.79,07/11/20,6947878
15,Lavavajillas aloe Bosque Verde concentrado,1,1.55,1.55,07/11/20,6947878
16,Lejía con detergente Bosque Verde,1,0.71,0.71,07/11/20,6947878


In [38]:
df['date'] = pd.to_datetime(df['date'])

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_12274/3532345252.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_12274/3532345252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


In [41]:
export_to_csv(df,"../../data/history-pruchase-palacet")

DataFrame successfully exported to ../data/history-pruchase-palacet.csv


In [40]:
df

,description,units,price_eu,unit_price,date,order_num
0,Friegasuelos pino Bosque Verde líquido,4,2.72,0.68,2020-09-16,6598904
1,Papel multiusos resistente Bosque Verde,1,2.89,2.89,2020-09-16,6598904
2,Papel higiénico doble rollo Bosque Verde,3,11.46,3.82,2020-09-16,6598904
3,Lavavajillas aloe Bosque Verde concentrado,4,6.20,1.55,2020-09-16,6598904
4,Suavizante ropa talco Bosque Verde concentrado,2,3.58,1.79,2020-09-16,6598904
...,...,...,...,...,...,...
13,Activador del lavado ropa blanca Bosque Verde ...,1,1.60,1.6,2020-07-11,6947878
14,Suavizante ropa talco Bosque Verde concentrado,2,3.58,1.79,2020-07-11,6947878
15,Lavavajillas aloe Bosque Verde concentrado,1,1.55,1.55,2020-07-11,6947878
16,Lejía con detergente Bosque Verde,1,0.71,0.71,2020-07-11,6947878
